# E-commerce Business Analytics Dashboard

A comprehensive analysis of e-commerce sales data focusing on business performance metrics, customer satisfaction, and operational efficiency.

## Table of Contents

1. [Introduction & Business Objectives](#introduction)
2. [Data Loading & Configuration](#data-loading)
3. [Data Dictionary](#data-dictionary)
4. [Data Preparation & Transformation](#data-preparation)
5. [Business Metrics Analysis](#business-metrics)
   - [Revenue Performance Analysis](#revenue-analysis)
   - [Product Category Performance](#product-analysis)
   - [Geographic Performance Analysis](#geographic-analysis)
   - [Customer Experience Analysis](#customer-analysis)
6. [Summary of Key Observations](#summary)

---

## 1. Introduction & Business Objectives {#introduction}

This analysis provides insights into e-commerce business performance through comprehensive examination of sales data. The primary objectives are:

- **Revenue Performance**: Analyze total revenue, growth trends, and order patterns
- **Product Strategy**: Identify top-performing categories and optimization opportunities
- **Geographic Insights**: Understand regional performance variations
- **Customer Satisfaction**: Evaluate delivery performance and review metrics
- **Operational Efficiency**: Assess delivery times and fulfillment quality

### Analysis Configuration

The analysis can be configured for different time periods by adjusting the parameters below:

In [ ]:
# Analysis Configuration
ANALYSIS_YEAR = 2023
COMPARISON_YEAR = 2022
ANALYSIS_MONTH = None  # Set to specific month (1-12) or None for full year
DATA_PATH = 'ecommerce_data/'

print(f"Analysis Period: {ANALYSIS_YEAR}")
print(f"Comparison Period: {COMPARISON_YEAR}")
if ANALYSIS_MONTH:
    print(f"Month Filter: {ANALYSIS_MONTH}")
else:
    print("Month Filter: Full Year")

## 2. Data Loading & Configuration {#data-loading}

Loading all required datasets and initializing the analysis framework.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_process_data
from business_metrics import (
    calculate_revenue_metrics, calculate_monthly_growth, calculate_order_metrics,
    calculate_product_category_sales, calculate_geographic_sales,
    calculate_delivery_performance, get_monthly_revenue_trend
)

# Configure display options
warnings.filterwarnings('ignore')
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("Libraries imported successfully")

In [ ]:
# Load and process all data
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data summary
data_summary = loader.get_data_summary()
print("Dataset Summary:")
print("=" * 50)
for dataset, info in data_summary.items():
    print(f"{dataset.upper()}:")
    print(f"  Rows: {info['rows']:,}")
    print(f"  Columns: {info['columns']}")
    print(f"  Memory: {info['memory_usage_mb']:.1f} MB")
    if info['date_range']:
        print(f"  Date Range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")
    print()

## 3. Data Dictionary {#data-dictionary}

### Key Business Terms and Column Definitions

| Column | Description | Business Impact |
|--------|-------------|----------------|
| **order_id** | Unique identifier for each customer order | Primary key for order-level analysis |
| **price** | Item price excluding shipping | Core revenue metric |
| **freight_value** | Shipping cost for the item | Additional revenue and cost analysis |
| **order_status** | Current status of the order | Operational efficiency indicator |
| **order_purchase_timestamp** | When the order was placed | Time-based analysis and trends |
| **order_delivered_customer_date** | When order was delivered to customer | Delivery performance metric |
| **product_category_name** | Product category classification | Product strategy and inventory planning |
| **customer_state** | Customer's state location | Geographic market analysis |
| **review_score** | Customer satisfaction rating (1-5) | Customer experience indicator |

### Calculated Metrics

- **Total Revenue**: Sum of all item prices for delivered orders
- **Average Order Value (AOV)**: Average total value per order
- **Delivery Days**: Time between order placement and delivery
- **Revenue Growth**: Year-over-year percentage change in revenue
- **Customer Satisfaction**: Distribution and average of review scores

## 4. Data Preparation & Transformation {#data-preparation}

Creating the comprehensive sales dataset for analysis with configurable time filters.

In [ ]:
# Create sales dataset for analysis period
sales_data = loader.create_sales_dataset(
    year_filter=ANALYSIS_YEAR,
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"Analysis Dataset Summary:")
print(f"Total Records: {len(sales_data):,}")
print(f"Unique Orders: {sales_data['order_id'].nunique():,}")
print(f"Date Range: {sales_data['order_purchase_timestamp'].min().date()} to {sales_data['order_purchase_timestamp'].max().date()}")
print(f"Total Revenue: ${sales_data['price'].sum():,.2f}")

# Display sample of the dataset
available_columns = ['order_id', 'price', 'purchase_year', 'purchase_month']
optional_columns = ['product_category_name', 'customer_state', 'review_score', 'delivery_days']

# Add optional columns if they exist
for col in optional_columns:
    if col in sales_data.columns:
        available_columns.append(col)

print(f"\nAvailable columns: {available_columns}")
print("\nSample Data:")
sales_data[available_columns].head()

In [ ]:
# Create comparison dataset if comparison year is specified
comparison_data = None
if COMPARISON_YEAR:
    comparison_data = loader.create_sales_dataset(
        year_filter=COMPARISON_YEAR,
        month_filter=ANALYSIS_MONTH,
        status_filter='delivered'
    )
    
    print(f"Comparison Dataset ({COMPARISON_YEAR}):")
    print(f"Total Records: {len(comparison_data):,}")
    print(f"Unique Orders: {comparison_data['order_id'].nunique():,}")
    print(f"Total Revenue: ${comparison_data['price'].sum():,.2f}")

# Create combined dataset for year-over-year analysis
if COMPARISON_YEAR:
    combined_data = loader.create_sales_dataset(
        month_filter=ANALYSIS_MONTH,
        status_filter='delivered'
    )
    # Filter to only include analysis and comparison years
    combined_data = combined_data[
        combined_data['purchase_year'].isin([ANALYSIS_YEAR, COMPARISON_YEAR])
    ]
else:
    combined_data = sales_data

## 5. Business Metrics Analysis {#business-metrics}

Comprehensive analysis of key business performance indicators.

### 5.1 Revenue Performance Analysis {#revenue-analysis}

Analyzing overall revenue trends, growth patterns, and key performance indicators.

In [ ]:
# Revenue metrics analysis
revenue_metrics = calculate_revenue_metrics(combined_data, ANALYSIS_YEAR, COMPARISON_YEAR)
order_metrics = calculate_order_metrics(combined_data, ANALYSIS_YEAR, COMPARISON_YEAR)

print(f"REVENUE PERFORMANCE ANALYSIS")
print(f"{'='*50}")
print(f"\n{ANALYSIS_YEAR} Performance:")
print(f"  Total Revenue: ${revenue_metrics['total_revenue_target']:,.2f}")
print(f"  Total Orders: {order_metrics['total_orders_target']:,}")
print(f"  Average Order Value: ${order_metrics['avg_order_value_target']:,.2f}")

if COMPARISON_YEAR:
    print(f"\n{COMPARISON_YEAR} Performance:")
    print(f"  Total Revenue: ${revenue_metrics['total_revenue_comparison']:,.2f}")
    print(f"  Total Orders: {order_metrics['total_orders_comparison']:,}")
    print(f"  Average Order Value: ${order_metrics['avg_order_value_comparison']:,.2f}")
    
    print(f"\nYear-over-Year Growth:")
    print(f"  Revenue Growth: {revenue_metrics['revenue_growth_percentage']:+.2f}%")
    print(f"  Order Growth: {order_metrics['orders_growth_percentage']:+.2f}%")
    print(f"  AOV Growth: {order_metrics['aov_growth_percentage']:+.2f}%")

    if revenue_metrics['revenue_growth_percentage'] > 0:
        print(f"\n✅ Revenue increased by ${revenue_metrics['total_revenue_target'] - revenue_metrics['total_revenue_comparison']:,.2f}")
    else:
        print(f"\n⚠️ Revenue decreased by ${revenue_metrics['total_revenue_comparison'] - revenue_metrics['total_revenue_target']:,.2f}")

In [ ]:
# Monthly revenue trend analysis
monthly_trend = get_monthly_revenue_trend(combined_data, ANALYSIS_YEAR)

# Visualize monthly revenue trend
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(monthly_trend['month'], monthly_trend['price'], 
        marker='o', linewidth=3, markersize=8, color='#1f77b4')

ax.set_title(f'Monthly Revenue Trend - {ANALYSIS_YEAR}', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Revenue ($)', fontsize=12)
ax.grid(True, alpha=0.3)

# Format y-axis as currency
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Add trend line
z = np.polyfit(monthly_trend['month'], monthly_trend['price'], 1)
p = np.poly1d(z)
ax.plot(monthly_trend['month'], p(monthly_trend['month']), 
        "--", alpha=0.7, color='red', label=f'Trend Line')

ax.legend()
plt.tight_layout()
plt.show()

# Monthly growth analysis
monthly_growth = calculate_monthly_growth(combined_data, ANALYSIS_YEAR)
avg_monthly_growth = monthly_growth.mean() * 100

print(f"\nMONTHLY TREND ANALYSIS - {ANALYSIS_YEAR}")
print(f"{'='*50}")
print(f"Average Monthly Growth Rate: {avg_monthly_growth:+.2f}%")

trend_slope = z[0]
trend_direction = "increasing" if trend_slope > 0 else "decreasing"
print(f"Revenue trend in {ANALYSIS_YEAR}: {trend_direction} (slope: ${trend_slope:,.0f}/month)")

### 5.2 Product Category Performance {#product-analysis}

Understanding which product categories drive the most revenue and identifying growth opportunities.

In [ ]:
# Product category analysis
if 'products' in processed_data:
    category_sales = calculate_product_category_sales(
        combined_data, processed_data['products'], ANALYSIS_YEAR
    )
    
    print(f"PRODUCT CATEGORY PERFORMANCE - {ANALYSIS_YEAR}")
    print(f"{'='*60}")
    print(f"\nTop 10 Categories by Revenue:")
    
    total_revenue = category_sales['price'].sum()
    for i, (_, row) in enumerate(category_sales.head(10).iterrows(), 1):
        revenue_share = (row['price'] / total_revenue) * 100
        print(f"{i:2d}. {row['product_category_name']:<25} ${row['price']:>10,.0f} ({revenue_share:5.1f}%)")
    
    print(f"\nCategory Performance Summary:")
    print(f"  Total Categories: {len(category_sales)}")
    print(f"  Top 3 Revenue Share: {(category_sales.head(3)['price'].sum() / total_revenue * 100):.1f}%")
    print(f"  Top 5 Revenue Share: {(category_sales.head(5)['price'].sum() / total_revenue * 100):.1f}%")
    
    # Visualize product category performance
    top_categories = category_sales.head(10)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.Blues_r(np.linspace(0.3, 0.8, len(top_categories)))
    bars = ax.barh(top_categories['product_category_name'], top_categories['price'], color=colors)
    
    ax.set_title(f'Top 10 Product Categories by Revenue - {ANALYSIS_YEAR}', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Revenue ($)', fontsize=12)
    ax.set_ylabel('Product Category', fontsize=12)
    
    # Format x-axis as currency
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # Add value labels on bars
    for i, (bar, value) in enumerate(zip(bars, top_categories['price'])):
        ax.text(value + value*0.01, bar.get_y() + bar.get_height()/2, 
                f'${value:,.0f}', va='center', ha='left', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("Product category data not available for analysis")

### 5.3 Geographic Performance Analysis {#geographic-analysis}

Analyzing sales performance across different geographic regions to identify market opportunities.

In [ ]:
# Geographic analysis
if 'customers' in loader.raw_data:
    geographic_sales = calculate_geographic_sales(
        combined_data, processed_data['orders'], loader.raw_data['customers'], ANALYSIS_YEAR
    )
    
    print(f"GEOGRAPHIC PERFORMANCE ANALYSIS - {ANALYSIS_YEAR}")
    print(f"{'='*60}")
    print(f"\nTop 15 States by Revenue:")
    
    total_geographic_revenue = geographic_sales['price'].sum()
    for i, (_, row) in enumerate(geographic_sales.head(15).iterrows(), 1):
        revenue_share = (row['price'] / total_geographic_revenue) * 100
        print(f"{i:2d}. {row['customer_state']:<3} ${row['price']:>10,.0f} ({revenue_share:5.1f}%)")
    
    print(f"\nGeographic Concentration:")
    print(f"  Total States: {len(geographic_sales)}")
    print(f"  Top 5 States Revenue Share: {(geographic_sales.head(5)['price'].sum() / total_geographic_revenue * 100):.1f}%")
    print(f"  Top 10 States Revenue Share: {(geographic_sales.head(10)['price'].sum() / total_geographic_revenue * 100):.1f}%")
    
    # Create geographic revenue heatmap
    fig = px.choropleth(
        geographic_sales,
        locations='customer_state',
        color='price',
        locationmode='USA-states',
        scope='usa',
        title=f'Revenue by State - {ANALYSIS_YEAR}',
        color_continuous_scale='Blues',
        labels={'price': 'Revenue ($)', 'customer_state': 'State'}
    )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        geo=dict(showframe=False, showcoastlines=True),
        height=500
    )
    
    fig.show()
else:
    print("Geographic data not available for analysis")

### 5.4 Customer Experience Analysis {#customer-analysis}

Evaluating customer satisfaction through review scores and delivery performance metrics.

In [ ]:
# Customer satisfaction and delivery analysis
if 'reviews' in loader.raw_data:
    delivery_metrics = calculate_delivery_performance(
        combined_data, loader.raw_data['reviews'], ANALYSIS_YEAR
    )
    
    print(f"CUSTOMER EXPERIENCE ANALYSIS - {ANALYSIS_YEAR}")
    print(f"{'='*60}")
    print(f"\nDelivery Performance:")
    print(f"  Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"\nCustomer Satisfaction:")
    print(f"  Average Review Score: {delivery_metrics['avg_review_score']:.2f}/5.0")
    
    print(f"\nDelivery Speed vs Satisfaction:")
    for category, score in delivery_metrics['delivery_categories_performance'].items():
        print(f"  {category:<10}: {score:.2f}/5.0")
    
    # Analyze review score distribution
    target_year_data = combined_data[combined_data['purchase_year'] == ANALYSIS_YEAR]
    review_data = target_year_data.dropna(subset=['review_score'])
    review_distribution = review_data['review_score'].value_counts().sort_index()
    
    print(f"\nReview Score Distribution - {ANALYSIS_YEAR}:")
    total_reviews = review_distribution.sum()
    for score, count in review_distribution.items():
        percentage = (count / total_reviews) * 100
        print(f"  {score} stars: {count:,} reviews ({percentage:.1f}%)")
    
    # Calculate satisfaction metrics
    high_satisfaction = (review_data['review_score'] >= 4).mean() * 100
    low_satisfaction = (review_data['review_score'] <= 2).mean() * 100
    
    print(f"\nSatisfaction Summary:")
    print(f"  High Satisfaction (4-5 stars): {high_satisfaction:.1f}%")
    print(f"  Low Satisfaction (1-2 stars): {low_satisfaction:.1f}%")
    
    # Visualize review score distribution and delivery performance
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Review score distribution
    colors = ['#d62728', '#ff7f0e', '#ffbb78', '#2ca02c', '#1f77b4']
    bars = ax1.bar(review_distribution.index, review_distribution.values, color=colors)
    ax1.set_title(f'Review Score Distribution - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Review Score (Stars)', fontsize=12)
    ax1.set_ylabel('Number of Reviews', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add percentage labels on bars
    for bar, count in zip(bars, review_distribution.values):
        percentage = (count / total_reviews) * 100
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + total_reviews*0.01,
                 f'{percentage:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # Delivery speed vs satisfaction
    delivery_categories = list(delivery_metrics['delivery_categories_performance'].keys())
    satisfaction_scores = list(delivery_metrics['delivery_categories_performance'].values())
    
    bars2 = ax2.bar(delivery_categories, satisfaction_scores, 
                    color=['#2ca02c', '#1f77b4', '#ff7f0e'])
    ax2.set_title(f'Customer Satisfaction by Delivery Speed - {ANALYSIS_YEAR}', 
                  fontsize=14, fontweight='bold')
    ax2.set_xlabel('Delivery Speed Category', fontsize=12)
    ax2.set_ylabel('Average Review Score', fontsize=12)
    ax2.set_ylim(0, 5)
    ax2.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars2, satisfaction_scores):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 f'{score:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Customer review data not available for analysis")

## 6. Summary of Key Observations {#summary}

### Executive Summary

Based on the comprehensive analysis of the e-commerce data, here are the key findings and recommendations:

In [ ]:
# Generate executive summary
print(f"EXECUTIVE BUSINESS SUMMARY - {ANALYSIS_YEAR}")
print("=" * 70)

print(f"\nRevenue Performance:")
print(f"  • Total Revenue: ${revenue_metrics['total_revenue_target']:,.0f}")
if COMPARISON_YEAR:
    print(f"  • YoY Growth: {revenue_metrics['revenue_growth_percentage']:+.1f}%")
    print(f"  • Monthly Avg Growth: {avg_monthly_growth:+.1f}%")

print(f"\nOrder Metrics:")
print(f"  • Total Orders: {order_metrics['total_orders_target']:,}")
print(f"  • Average Order Value: ${order_metrics['avg_order_value_target']:,.0f}")
if COMPARISON_YEAR:
    print(f"  • Order Growth: {order_metrics['orders_growth_percentage']:+.1f}%")

if 'products' in processed_data:
    print(f"\nProduct Performance:")
    top_category = category_sales.iloc[0]
    top_category_share = (top_category['price'] / total_revenue) * 100
    print(f"  • Top Category: {top_category['product_category_name']}")
    print(f"  • Top Category Revenue: ${top_category['price']:,.0f} ({top_category_share:.1f}%)")
    print(f"  • Total Categories: {len(category_sales)}")

if 'customers' in loader.raw_data:
    print(f"\nGeographic Performance:")
    top_state = geographic_sales.iloc[0]
    top_state_share = (top_state['price'] / total_geographic_revenue) * 100
    print(f"  • Top State: {top_state['customer_state']}")
    print(f"  • Top State Revenue: ${top_state['price']:,.0f} ({top_state_share:.1f}%)")
    print(f"  • Active States: {len(geographic_sales)}")

if 'reviews' in loader.raw_data:
    print(f"\nCustomer Experience:")
    print(f"  • Average Review Score: {delivery_metrics['avg_review_score']:.2f}/5.0")
    print(f"  • Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"  • High Satisfaction Rate: {high_satisfaction:.1f}%")

print(f"\n{'='*70}")

In [ ]:
# Generate insights and recommendations based on the analysis
insights = []
recommendations = []

# Revenue insights
if COMPARISON_YEAR:
    if revenue_metrics['revenue_growth_percentage'] > 0:
        insights.append(f"✓ Revenue grew by {revenue_metrics['revenue_growth_percentage']:.1f}% year-over-year, indicating positive business momentum")
    else:
        insights.append(f"⚠ Revenue declined by {abs(revenue_metrics['revenue_growth_percentage']):.1f}% year-over-year, requiring attention")
        recommendations.append("Focus on customer acquisition and retention strategies to reverse revenue decline")

    # Order insights
    if order_metrics['orders_growth_percentage'] > 0:
        insights.append(f"✓ Order volume increased by {order_metrics['orders_growth_percentage']:.1f}%, showing customer acquisition success")
    else:
        insights.append(f"⚠ Order volume decreased by {abs(order_metrics['orders_growth_percentage']):.1f}%, indicating customer retention challenges")
        recommendations.append("Implement customer retention programs and improve order conversion rates")

# Monthly trend insights
if avg_monthly_growth > 0:
    insights.append(f"✓ Positive average monthly growth of {avg_monthly_growth:.1f}% shows consistent improvement")
else:
    insights.append(f"⚠ Negative average monthly growth of {avg_monthly_growth:.1f}% suggests declining momentum")
    recommendations.append("Analyze monthly performance patterns and adjust marketing strategies accordingly")

# Customer satisfaction insights
if 'reviews' in loader.raw_data:
    if delivery_metrics['avg_review_score'] >= 4.0:
        insights.append(f"✓ High customer satisfaction ({delivery_metrics['avg_review_score']:.2f}/5.0) indicates strong service quality")
    elif delivery_metrics['avg_review_score'] >= 3.5:
        insights.append(f"⚠ Moderate customer satisfaction ({delivery_metrics['avg_review_score']:.2f}/5.0) has room for improvement")
        recommendations.append("Implement customer feedback programs to address satisfaction gaps")
    else:
        insights.append(f"⚠ Low customer satisfaction ({delivery_metrics['avg_review_score']:.2f}/5.0) requires immediate attention")
        recommendations.append("Prioritize customer experience improvements and service quality enhancements")

    # Delivery insights
    if delivery_metrics['avg_delivery_days'] <= 5:
        insights.append(f"✓ Fast delivery performance ({delivery_metrics['avg_delivery_days']:.1f} days average) supports customer satisfaction")
    else:
        insights.append(f"⚠ Delivery time ({delivery_metrics['avg_delivery_days']:.1f} days average) may impact customer satisfaction")
        recommendations.append("Optimize logistics and delivery processes to reduce average delivery time")

# Geographic concentration insights
if 'customers' in loader.raw_data:
    top_5_concentration = (geographic_sales.head(5)['price'].sum() / total_geographic_revenue * 100)
    if top_5_concentration > 50:
        insights.append(f"⚠ High geographic concentration ({top_5_concentration:.1f}% in top 5 states) creates market risk")
        recommendations.append("Develop expansion strategies in underrepresented geographic markets")
    else:
        insights.append(f"✓ Good geographic diversification ({top_5_concentration:.1f}% in top 5 states) reduces market risk")

# Product concentration insights
if 'products' in processed_data:
    top_3_product_concentration = (category_sales.head(3)['price'].sum() / total_revenue * 100)
    if top_3_product_concentration > 70:
        insights.append(f"⚠ High product category concentration ({top_3_product_concentration:.1f}% in top 3 categories)")
        recommendations.append("Diversify product portfolio to reduce dependency on top-performing categories")

print("KEY BUSINESS INSIGHTS:")
print("=" * 50)
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\nSTRATEGIC RECOMMENDATIONS:")
print("=" * 50)
if recommendations:
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
else:
    print("✓ Business performance appears strong across all analyzed metrics")

# Additional general recommendations
general_recommendations = [
    "Monitor monthly trends closely to identify seasonal patterns and adjust inventory accordingly",
    "Consider implementing A/B testing for pricing and product positioning strategies",
    "Develop targeted marketing campaigns for high-performing geographic regions",
    "Establish regular customer feedback collection and analysis processes"
]

print("\nGENERAL RECOMMENDATIONS:")
print("=" * 50)
for i, rec in enumerate(general_recommendations, 1):
    print(f"{i}. {rec}")

---

## Analysis Configuration Summary

This analysis was configured for:
- **Target Year**: 2023
- **Comparison Year**: 2022
- **Analysis Period**: Full year
- **Order Status Filter**: Delivered orders only

To modify the analysis parameters, update the configuration section at the beginning of this notebook and re-run all cells.

### Framework Benefits

This refactored notebook provides:

1. **Modular Structure**: Clean separation of data loading, metrics calculation, and visualization
2. **Configurable Analysis**: Easy modification of time periods and filters
3. **Reusable Functions**: Business metrics functions can be used across different analyses
4. **Comprehensive Coverage**: Revenue, product, geographic, and customer experience analysis
5. **Actionable Insights**: Data-driven recommendations for business decisions
6. **Professional Documentation**: Clear explanations and business context

### Next Steps

1. Schedule regular execution of this analysis (monthly/quarterly)
2. Create automated reports using the business_metrics.py functions
3. Implement additional metrics as business needs evolve
4. Develop predictive analytics using historical trends
5. Create real-time dashboards for ongoing monitoring

**Note**: This notebook uses modular functions from `business_metrics.py` and `data_loader.py` for maintainable and reusable analytics code.